# ASPerf performance analysis

We ran the problem instance as described in the formulation notebook but included the --stats flag

Importantly, the program takes a significant amount of time to ground (~200 seconds) and although it is able to quickly find models after that it takes a significant amount of time to make progress after a certain point.

In a similar test we left an instance of this problem running for more than a day and it did not progress beyond the bounds we have shown.

In [10]:
!tail -n 50 test_stat

hash(22345) = 34
hash(22358) = 51
hash(22361) = 65
Answer: 8
edge(0,1) edge(0,2) edge(1,3) edge(1,4) op(0,mod) op(2,const) op(1,mul) op(3,key) op(4,const)
Optimization: 194 5
^C
*** Info : (clingo): Sending shutdown signal...
*** ERROR: (clingo): <python>: error: error calling main function:
  Traceback (most recent call last):
    File "<verify.lp:117:1-189:6>", line 4, in main
  RuntimeError: solving stopped by signal

*** Info : (clingo): Shutdown completed in 0.010 seconds
SATISFIABLE

INTERRUPTED  : 1
Models       : 8+
  Optimum    : unknown
Optimization : 194 5
Calls        : 1
Time         : 1365.827s (Solving: 1140.09s 1st Model: 4.79s Unsat: 0.00s)
CPU Time     : 1365.200s

Choices      : 1591558 
Conflicts    : 169531   (Analyzed: 169531)
Restarts     : 508      (Average: 333.72 Last: 3959)
Model-Level  : 33.8    
Problems     : 1        (Average Length: 1.00 Splits: 0)
Lemmas       : 169531   (Deleted: 118394)
  Binary     : 3998     (Ratio:   2.36%)
  Ternary    : 9211     

# Worst grounding rules

In general, the bulk of our grounded output comes from val_bit(K,I,B,V) rules, which make up the core of our hash function evaluation. Looking at an example of a typical val_bit rule we can see that the rule is quadratic in the number of nodes I\* , linear in the number of keys, but thankfully only logarithmic in the range of values due to our bit encoding. An example rule is shown below:

val_bit(K,I,B,V1^V2) :- 
    op(I,xor),
    edge(I,I1),
    edge(I,I2),
    I1 < I2, 
    val_bit(K,I1,B,V1), 
    val_bit(K,I2,B,V2).
    
Alternatively a val_bit is obtained from a yosys circuit

val_bit(K,I,B,V) :- output_signal(I,K,B,V).

Where the setup required to specify the inputs to the circuit is similar in complexity to the bitwise val_bit operations. It should be noted that when dealing with circuits for binary operations the grounding time becomes dominated by the rules related to them (notably, variants of signal) due to the large amount of grounded output needed to represent each operation.

With the template function applied as in this case we used the --text option to ground we can see that the grounded output is overwhelmingly dedicated to representing yosys circuits. The second largest thing represented is hash_bit_agree, defined as 

hash_bit(K,B,V) :- val_bit(K,0,B,V).

hash_bit_agree(K1,K2,B) :- K1 < K2, hash_bit(K1,B,V), hash_bit(K2,B,V).
    
collision(K1,K2) :- 
    key(K1);
    key(K2); 
    hash_bit_agree(K1,K2,B):B=0..bits-1.

:- collision(K1,K2).

which is notably needed for checking for collisions.

In [ ]:
!cat states_ground_template | sed -E 's/[0-9]+/D/g' > out

In [13]:
!cat out | sort | uniq -c | sort -n

      1 bin_op(and).
      1 bin_op(mod).
      1 bin_op(mul).
      1 bin_op(sub).
      1 bin_op(xor).
      1 bits(D).
      1 #delayed(D) <=> D<=#count{D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,worst_key(D):worst_key(D);D,wo